<a href="https://colab.research.google.com/github/PatrykCiwinski/NLP_to_SQL/blob/main/sql_openAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##Necessary imports

In [ ]:
import openai

In [ ]:
from dotenv import dotenv_values

In [ ]:
config = dotenv_values('.env')

In [ ]:
openai.api_key=config['OPENAI_API_KEY']

In [ ]:
import pandas as pd

In [ ]:
##Creating data frame

In [ ]:
df = pd.read_csv('sales_data_sample.csv', encoding='cp437')

In [ ]:
##Connecting to date base , which is crated in memory.

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy import text

In [ ]:
import sqlalchemy

In [ ]:
temporary_db = create_engine('sqlite:///:memory:',echo=True)

In [ ]:
data = df.to_sql('Sales',con=temporary_db)

In [ ]:
with temporary_db.connect() as conn:
    result=conn.execute(text('SELECT * from Sales'))

In [ ]:
##We are showing the definition of table, so openai is aware of it

In [ ]:
def create_definition(db):
    prompt="""
    #Sales({})
    """.format(','.join(str(col)for col in result.keys()))
    return prompt

In [ ]:
##Now we are combining this definition and input from user

In [ ]:
def prompt_input():
    nlp_text = input('What do you want to do?: ')
    return nlp_text

In [ ]:
def combine(df,prompt_input):
    definition = create_definition(df)
    query=f'Query to answer: {prompt_input()}'
    return definition+query

In [ ]:
response=openai.Completion.create(
model='text-davinci-003',
prompt=combine(df,prompt_input),
temperature=0,
max_tokens=150,
frequency_penalty=0,
presence_penalty=0,
stop=['#',';'],
n=1)

In [ ]:
response

In [ ]:
##Now we have response from openai model, we can apply it (return sql query function) on our table

In [ ]:
def return_sql_query(response):
    query=response['choices'][0]['text']
    if query.startswith(' '):
        query='SELECT'+query
    return query

In [ ]:
with temporary_db.connect() as conn:
    result = conn.execute(text(return_sql_query(response)))

In [ ]:
##We can see results from applied query

In [ ]:
result.all()